In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)

## Read Data

In [3]:
DATA_DIRECTORY = '../data'
PBP_PATH = os.path.join(DATA_DIRECTORY, "play_by_play", '2020.csv')

In [148]:
df = pd.read_csv(PBP_PATH, low_memory=False)
df.shape

(2900, 340)

In [149]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,...,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weath

In [176]:
df_rush_pbp = df.loc[
    (((df['play_type'] == 'run') | (df['play_type'] == 'qb_kneel'))
    & (df['sack'] == 0)
    & (df["two_point_attempt"] == 0) 
    # & (df["qb_kneel"] == 0))
    )
]

In [164]:
df_rush_pbp['rush'].value_counts()

1    820
0     85
Name: rush, dtype: int64

In [165]:
df_rush_pbp['pass'].value_counts()

0    841
1     64
Name: pass, dtype: int64

In [166]:
df_rush_pbp['play_type'].value_counts()

run         884
qb_kneel     21
Name: play_type, dtype: int64

In [167]:
df_rush_pbp['passer'].value_counts()

K.Murray            7
G.Minshew II        4
T.Bridgewater       4
K.Cousins           4
J.Burrow            4
D.Watson            4
M.Stafford          4
J.Allen             3
D.Prescott          3
R.Fitzpatrick       3
D.Jones             3
M.Trubisky          2
L.Jackson           2
J.Goff              2
T.Taylor            2
D.Haskins           2
D.Brees             1
R.Tannehill         1
R.Wilson            1
T.Hill              1
P.Rivers            1
D.Lock              1
T.Brady             1
B.Mayfield          1
J.Garoppolo         1
B.Roethlisberger    1
C.Newton            1
Name: passer, dtype: int64

In [168]:
sum(df_rush_pbp['passer'].notnull())

64

In [169]:
debug = df_rush_pbp.loc[df_rush_pbp['rush'] == 0]
debug['is_scramble'] = debug['desc'].apply(lambda x: "scramble" in x)
debug['is_scramble'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


True     64
False    21
Name: is_scramble, dtype: int64

In [180]:
cols = ["rusher", "play_type", "qb_kneel", "rush", "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

df_rush_stats = df_rush_pbp[cols].groupby(["rusher", "play_type"], as_index=False).sum()
df_rush_stats.sort_values("yards_gained", ascending=False)

,rusher,play_type,qb_kneel,rush,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
23,C.Edwards-Helaire,run,0,25,138.0,1.0,0.0,0.0,0.0
40,D.Henry,run,0,31,116.0,0.0,0.0,0.0,0.0
17,B.Snell,run,0,19,113.0,0.0,1.0,0.0,0.0
25,C.McCaffrey,run,0,23,96.0,2.0,0.0,0.0,0.0
53,E.Elliott,run,0,22,96.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
58,G.Minshew II,qb_kneel,1,0,-3.0,0.0,0.0,0.0,0.0
135,T.Montgomery,run,0,1,-4.0,0.0,0.0,0.0,0.0
109,P.Ricard,run,0,1,-5.0,0.0,1.0,1.0,0.0
37,D.Haskins,qb_kneel,3,0,-6.0,0.0,0.0,0.0,0.0


In [178]:
df_rush_stats['play_type'].value_counts()

run         124
qb_kneel     16
Name: play_type, dtype: int64

In [172]:
cols = ["passer", "play_type", 'pass', "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

df_rush_stats_scrambles = df_rush_pbp[cols].groupby(["passer", "play_type"], as_index=False).sum()
df_rush_stats_scrambles.sort_values("yards_gained", ascending=False)

,passer,play_type,pass,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
15,K.Murray,run,7,90.0,1.0,0.0,0.0,0.0
14,K.Cousins,run,4,34.0,0.0,0.0,0.0,0.0
7,D.Prescott,run,3,30.0,0.0,0.0,0.0,0.0
10,J.Allen,run,3,28.0,0.0,0.0,0.0,0.0
8,D.Watson,run,4,27.0,0.0,0.0,0.0,0.0
24,T.Bridgewater,run,4,26.0,0.0,0.0,0.0,0.0
18,M.Trubisky,run,2,26.0,0.0,0.0,0.0,0.0
5,D.Jones,run,3,23.0,0.0,0.0,0.0,0.0
16,L.Jackson,run,2,22.0,0.0,0.0,0.0,0.0
9,G.Minshew II,run,4,22.0,0.0,0.0,0.0,0.0


array(['run'], dtype=object)

## Debugging QB rushes

In [82]:
df_rush_stats[df_rush_stats['rusher'] == 'K.Murray']

,rusher,rush,rush_attempt,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
78,K.Murray,4,4.0,10.0,0.0,0.0,0.0,0.0


In [22]:
cols = [col for col in df if "fumble" in col]
cols

['fumble_forced',
 'fumble_not_forced',
 'fumble_out_of_bounds',
 'fumble_lost',
 'fumble',
 'forced_fumble_player_1_team',
 'forced_fumble_player_1_player_id',
 'forced_fumble_player_1_player_name',
 'forced_fumble_player_2_team',
 'forced_fumble_player_2_player_id',
 'forced_fumble_player_2_player_name',
 'fumbled_1_team',
 'fumbled_1_player_id',
 'fumbled_1_player_name',
 'fumbled_2_player_id',
 'fumbled_2_player_name',
 'fumbled_2_team',
 'fumble_recovery_1_team',
 'fumble_recovery_1_yards',
 'fumble_recovery_1_player_id',
 'fumble_recovery_1_player_name',
 'fumble_recovery_2_team',
 'fumble_recovery_2_yards',
 'fumble_recovery_2_player_id',
 'fumble_recovery_2_player_name']

In [33]:
cols = [col for col in df if "kneel" in col]
cols

['qb_kneel']

In [54]:
debug = df[(df['play_type'] == 'pass') & (df['rusher'].isna())]
debug.shape

(1192, 340)

In [56]:
sorted(list(df.columns))

['aborted_play',
 'air_epa',
 'air_wpa',
 'air_yards',
 'assist_tackle',
 'assist_tackle_1_player_id',
 'assist_tackle_1_player_name',
 'assist_tackle_1_team',
 'assist_tackle_2_player_id',
 'assist_tackle_2_player_name',
 'assist_tackle_2_team',
 'assist_tackle_3_player_id',
 'assist_tackle_3_player_name',
 'assist_tackle_3_team',
 'assist_tackle_4_player_id',
 'assist_tackle_4_player_name',
 'assist_tackle_4_team',
 'away_coach',
 'away_score',
 'away_team',
 'away_timeouts_remaining',
 'away_wp',
 'away_wp_post',
 'blocked_player_id',
 'blocked_player_name',
 'comp_air_epa',
 'comp_air_wpa',
 'comp_yac_epa',
 'comp_yac_wpa',
 'complete_pass',
 'cp',
 'cpoe',
 'def_wp',
 'defensive_extra_point_attempt',
 'defensive_extra_point_conv',
 'defensive_two_point_attempt',
 'defensive_two_point_conv',
 'defteam',
 'defteam_score',
 'defteam_score_post',
 'defteam_timeouts_remaining',
 'desc',
 'div_game',
 'down',
 'drive',
 'drive_end_transition',
 'drive_end_yard_line',
 'drive_ended_with_

In [57]:
df['pass_attempt'].unique()

array([nan,  0.,  1.])

In [58]:
df['pass'].unique()

array([0, 1])

In [61]:
df[(df['pass_attempt']==1) & (df['pass'] == 0)].head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,...,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weath

In [71]:
debug = df[(df['pass_attempt']==0) & (df['pass'] == 1)]
debug = debug[debug['play_type'] != 'no_play']
debug.shape
#debug['desc'].head()

(64, 340)

In [72]:
debug['play_type'].value_counts()

run    64
Name: play_type, dtype: int64

In [73]:
debug['rush_attempt'].value_counts()

1.0    64
Name: rush_attempt, dtype: int64

In [74]:
debug['rusher'].value_counts()

Series([], Name: rusher, dtype: int64)

In [75]:
debug['passer'].value_counts()

K.Murray            7
K.Cousins           4
M.Stafford          4
G.Minshew II        4
D.Watson            4
T.Bridgewater       4
J.Burrow            4
R.Fitzpatrick       3
D.Prescott          3
D.Jones             3
J.Allen             3
L.Jackson           2
J.Goff              2
M.Trubisky          2
D.Haskins           2
T.Taylor            2
B.Mayfield          1
T.Brady             1
P.Rivers            1
R.Tannehill         1
D.Brees             1
R.Wilson            1
B.Roethlisberger    1
C.Newton            1
J.Garoppolo         1
D.Lock              1
T.Hill              1
Name: passer, dtype: int64

In [76]:
debug['sack'].value_counts()

0.0    64
Name: sack, dtype: int64

In [77]:
debug['desc'].head(20)

36     (4:29) (No Huddle, Shotgun) 1-K.Murray scrambl...
101    (11:37) (Shotgun) 1-K.Murray scrambles left en...
103    (10:42) (Shotgun) 1-K.Murray scrambles right e...
111    (8:10) 1-K.Murray scrambles right end to ARI 2...
137    (13:55) (Shotgun) 1-K.Murray scrambles right e...
144    (10:35) (No Huddle, Shotgun) 1-K.Murray scramb...
161    (7:22) (No Huddle, Shotgun) 1-K.Murray scrambl...
170    (3:59) 10-J.Garoppolo scrambles up the middle ...
252    (7:32) 9-M.Stafford scrambles up the middle to...
270    (1:23) (Shotgun) 9-M.Stafford scrambles up the...
319    (1:04) (Shotgun) 10-M.Trubisky scrambles up th...
339    (10:44) (Shotgun) 10-M.Trubisky scrambles left...
371    (1:54) (Shotgun) 9-M.Stafford scrambles up the...
373    (1:12) (No Huddle, Shotgun) 9-M.Stafford scram...
403    (7:41) 6-B.Mayfield scrambles left end to CLE ...
423    (:20) 8-L.Jackson scrambles right end ran ob a...
522    (13:52) (Shotgun) 8-L.Jackson scrambles right ...
621    (7:43) (Shotgun) 4-D.Pre

In [80]:
debug[debug['desc'].str.contains('scramble')].shape

(64, 340)

In [88]:
debug = df[(df['rusher'] == 'K.Murray') & (df["qb_kneel"] == 0)]
debug['desc'].values

array(['(5:16) (Shotgun) 1-K.Murray right tackle to ARI 23 for 9 yards (41-E.Moseley).',
       '(:18) (Shotgun) 1-K.Murray left end to SF 43 for no gain (41-E.Moseley).',
       '(2:11) (No Huddle, Shotgun) 1-K.Murray up the middle to ARI 6 for no gain (24-K.Williams).',
       '(7:48) (No Huddle, Shotgun) 1-K.Murray up the middle to ARI 47 for 1 yard (90-K.Givens).'],
      dtype=object)

In [102]:
debug = df[(df['passer'] == 'K.Murray') & (df["qb_kneel"] == 0) & (df['play_type'] != 'no_play')]
debug['play_type'].value_counts()

pass    41
run      7
Name: play_type, dtype: int64

In [118]:
debug = df[(df['passer'] == 'K.Murray') & (df['play_type'] != 'no_play')]

cols = ["passer", "play_type", "rush_attempt", "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

df_rush_stats = debug[cols].groupby(["passer", 'play_type'], as_index=False).sum()
df_rush_stats.sort_values("yards_gained", ascending=False).head(25)

,passer,play_type,rush_attempt,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
0,K.Murray,pass,0.0,224.0,0.0,0.0,0.0,0.0
1,K.Murray,run,7.0,90.0,1.0,0.0,0.0,0.0


In [174]:
debug = df[(df['rusher'] == 'K.Murray') & (df['play_type'] != 'no_play')]

cols = ["rusher", 'qb_kneel', "play_type", "rush_attempt", "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

df_rush_stats = debug[cols].groupby(["rusher", 'play_type'], as_index=False).sum()
df_rush_stats.sort_values("yards_gained", ascending=False).head(25)

,rusher,play_type,qb_kneel,rush_attempt,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
1,K.Murray,run,0,4.0,10.0,0.0,0.0,0.0,0.0
0,K.Murray,qb_kneel,2,2.0,-9.0,0.0,0.0,0.0,0.0


In [103]:
debug['rush'].value_counts()

0    48
Name: rush, dtype: int64

In [104]:
debug['pass'].value_counts()

1    48
Name: pass, dtype: int64

In [105]:
debug['rush_attempt'].value_counts()

0.0    41
1.0     7
Name: rush_attempt, dtype: int64

In [106]:
debug['pass_attempt'].value_counts()

1.0    41
0.0     7
Name: pass_attempt, dtype: int64

In [109]:
debug = df[(df['rusher'] == 'K.Murray') & (df["qb_kneel"] == 0) & (df['play_type'] != 'no_play')]
debug['play_type'].value_counts()

run    4
Name: play_type, dtype: int64

In [110]:
debug['rush'].value_counts()

1    4
Name: rush, dtype: int64

In [111]:
debug['pass'].value_counts()

0    4
Name: pass, dtype: int64